In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
import haversine as hs
import scipy.stats as st
import seaborn as sns
sns.set()

from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from faker import Faker

pio.renderers.default = "notebook"

## Suppress scientific notation
pd.options.display.float_format = '{:.4f}'.format

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./dataset.csv')
df.head()

,DriverId,EventName,Latitude,Longitude,Speed km/h,ts
0,0,Timed Event,34.1866,-118.0881,64.0000,2017-11-01 00:00:02.430
1,0,Distance Event,34.1861,-118.0892,53.0000,2017-11-01 00:00:05.600
2,0,Distance Event,34.1864,-118.0896,34.0000,2017-11-01 00:00:13.640
3,0,Distance Event,34.1875,-118.0889,33.0000,2017-11-01 00:00:26.070
4,0,Distance Event,34.1887,-118.0865,32.0000,2017-11-01 00:00:35.090


In [3]:
df['ts'] = pd.to_datetime(df['ts'])
df.head()

,DriverId,EventName,Latitude,Longitude,Speed km/h,ts
0,0,Timed Event,34.1866,-118.0881,64.0000,2017-11-01 00:00:02.430
1,0,Distance Event,34.1861,-118.0892,53.0000,2017-11-01 00:00:05.600
2,0,Distance Event,34.1864,-118.0896,34.0000,2017-11-01 00:00:13.640
3,0,Distance Event,34.1875,-118.0889,33.0000,2017-11-01 00:00:26.070
4,0,Distance Event,34.1887,-118.0865,32.0000,2017-11-01 00:00:35.090


In [7]:
eventsPerDriver = df.groupby('DriverId').agg({"EventName":"count"}).add_suffix('_count')
averageNoEvents = np.mean(eventsPerDriver).values[0].round(1)
averageNoEvents

10108.0

In [8]:
eventsPerDriver.head()

,EventName_count
DriverId,
0,17345
1,20694
2,11945
3,819
4,6811


In [10]:
eventsPerDriver.sort_values(by='EventName_count', ascending=False, inplace=True)
eventsPerDriver.head()

,EventName_count
DriverId,
49,31942
77,30420
45,30319
51,26917
5,24152


In [11]:
fig = px.histogram(eventsPerDriver, x='EventName_count', color='driverId', title='Histogram of driverId by Number of Events')

fig.show()

ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['EventName_count'] but received: driverId